In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
df.reset_index(inplace=True)
df

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [3]:
measure_features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked']
labels = ['Survived']

train_df_x = df[measure_features]
train_df_y = df[labels]

In [4]:
clean_enc_dic = {'Sex': {'male':0, 'female':1}, 'Embarked': {'S': 0, 'C':1, 'Q':2}}
train_df_x.replace(clean_enc_dic, inplace=True)
train_df_x

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


,Pclass,Sex,Age,SibSp,Parch,Embarked
0,3,0,22.0,1,0,0.0
1,1,1,38.0,1,0,1.0
2,3,1,26.0,0,0,0.0
3,1,1,35.0,1,0,0.0
4,3,0,35.0,0,0,0.0
...,...,...,...,...,...,...
886,2,0,27.0,0,0,0.0
887,1,1,19.0,0,0,0.0
888,3,1,NaN,1,2,0.0
889,1,0,26.0,0,0,1.0


In [5]:
train_df_x.isna().sum()

Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [6]:
for x in ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']:
    train_df_x[x].replace(np.NaN, train_df_x[x].median(), inplace=True)

train_df_x['Age'].replace(np.NaN, train_df_x['Age'].mean(), inplace=True)
    
train_df_x.isnull().sum()

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [7]:
from sklearn.svm import SVC
model = SVC(kernel='linear')
model.fit(train_df_x, train_df_y)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
test_df = pd.read_csv('/kaggle/input/titanic/test.csv')

test_df.reset_index(inplace=True)

test_df_x = test_df[measure_features]

test_df_x.replace(clean_enc_dic, inplace=True)

for x in ['Pclass', 'Sex', 'SibSp', 'Parch', 'Embarked']:
    test_df_x[x].replace(np.NaN, test_df_x[x].median(), inplace=True)

test_df_x['Age'].replace(np.NaN, test_df_x['Age'].mean(), inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6702: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [9]:
predict_y = model.predict(test_df_x)
len(predict_y)

418

In [10]:
output = pd.DataFrame({'PassengerId': test_df.PassengerId, 'Survived':predict_y})
output.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [11]:
output.to_csv('my_submission.csv', index=False)
print("Submission Saved")

Submission Saved
